# Plotting with Atlite

This little notebook creates all the plots given in the introduction section. Mainly using the packages [Cartopy](https://pypi.org/project/Cartopy/), [GeoPandas](http://geopandas.org/) and [xarray](http://xarray.pydata.org/en/stable/) geographical plotting can be efficiently and straightfowardly done. Since they interact well together, one has just to consider some essential commands. In particular the package scopes are distributed as following 

* when it comes to projections, ask Cartopy (projection, transformation)
* when it comes to plotting shapes, ask GeoPandas
* when it comes to plotting data on geographical grids, ask xarray

So, let's directly go to the code!

First of all import all relevant packages 

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import seaborn as sns
import cartopy.crs as ccrs
from cartopy.crs import PlateCarree as plate
import cartopy.io.shapereader as shpreader
import xarray as xs
from matplotlib.gridspec import GridSpec
import atlite
import os

## Create Shapes for United Kingdom and Ireland
* use the shapereader of Cartopy to retrieve high resoluted shapes
* make a GeoSeries with the shapes

In [ ]:
shpfilename = shpreader.natural_earth(resolution='10m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
UkIr = gpd.GeoSeries({r.attributes['NAME_EN']: r.geometry
                    for r in reader.records()}, crs={'init' :'epsg:4326'})\
                    .reindex(['United Kingdom', 'Ireland'])

### Create Cutout
* create a cutout with geographical bounds of the shapes

In [ ]:

# Define the cutout; this will not yet trigger any major operations
cutout = atlite.Cutout(name="uk-2011-01",
                       cutout_dir="./",
                       module="era5",
                       bounds=UkIr.unary_union.bounds,
                       time="2011-01")

# This is where all the work happens (this can take some time).
cutout.prepare()


## Define a overall projection 
This projection will be used throughout the following plots. It has to be assigned to every axis that should be based on this projection

In [ ]:
projection = ccrs.Orthographic(-10, 35)

## Plot Earth with cutout bound
* create GeoSeries with cell relevant data
* plot 'naturalearth_lowres' (country shapes) with unary union of cells on top

In [ ]:
cells = gpd.GeoDataFrame({'geometry':cutout.grid_cells,
                          'capfactors':None,
                          'x': cutout.grid_coordinates()[:,0],
                          'y': cutout.grid_coordinates()[:,1]})

df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
country_bound = gpd.GeoSeries(cells.unary_union)

projection = ccrs.Orthographic(-10, 35)
fig, ax = plt.subplots(subplot_kw={'projection':projection}, figsize=(6,6))
df.plot(ax=ax, transform=plate())
country_bound.plot(ax=ax, edgecolor='orange', facecolor='None', transform=plate())
fig.tight_layout()

## Plot Cutout with Solar Influx Data
* create matplotlib [GridSpec](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.gridspec.GridSpec.html#matplotlib.gridspec.GridSpec)
* country shapes and cells on left hand side
* time series of wind100m, influx_direct, runoff on right hand side

In [ ]:
fig = plt.figure(figsize=(12,7))
gs = GridSpec(3, 3, figure=fig)

ax = fig.add_subplot(gs[:, 0:2], projection=projection)
plot_grid_dict = dict(alpha=0.1, edgecolor='k', zorder=4, facecolor='None', transform=plate())
UkIr.plot(ax=ax, zorder=1, transform=plate())
cells.plot(ax=ax, **plot_grid_dict)
country_bound.plot(ax=ax, edgecolor='orange', facecolor='None', transform=plate())
ax.outline_patch.set_edgecolor('white')

ax1 = fig.add_subplot(gs[0,2])
cutout.data.wnd100m.mean(['x', 'y']).plot(ax=ax1)
ax1.set_frame_on(False)
ax1.xaxis.set_visible(False)

ax2 = fig.add_subplot(gs[1,2], sharex=ax1)
cutout.data.influx_direct.mean(['x', 'y']).plot(ax=ax2)
ax2.set_frame_on(False)
ax2.xaxis.set_visible(False)

ax3 = fig.add_subplot(gs[2,2], sharex=ax1)
cutout.data.runoff.mean(['x', 'y']).plot(ax=ax3)
ax3.set_frame_on(False)
ax3.set_xlabel(None)
fig.tight_layout()

## Plot Capacity Factors
* use xarray plotting function to directly plot data
* plot cells GeoSeries on top

In [ ]:
cap_factors = cutout.wind(turbine='Vestas_V112_3MW', capacity_factor=True)

fig, ax = plt.subplots(subplot_kw={'projection':projection}, figsize=(9,7))
cap_factors.name = 'Capacity Factor'
cap_factors.plot(ax=ax, transform=plate(), alpha=0.8)
cells.plot(ax=ax, **plot_grid_dict)
ax.outline_patch.set_edgecolor('white')
fig.tight_layout()

In [ ]:
# Define capacity layout on plot it

In [ ]:
sites = gpd.GeoDataFrame([['london', 0.7, 51.3, 20],
                      ['dublin', -6.16, 53.21, 30],
                      ['edinburgh', -3.13, 55.5, 10]],
                     columns=['name', 'x', 'y', 'capacity']).set_index('name')

nearest = cutout.data.sel({'x':sites.x.values, 'y':sites.y.values}, 'nearest').coords
sites['x'] = nearest.get('x').values
sites['y'] = nearest.get('y').values
cells_production = sites.merge(cells, how='inner').rename(pd.Series(sites.index))

layout = xs.DataArray(cells_production.set_index(['y', 'x']).capacity.unstack())\
                    .reindex_like(cap_factors).rename('Installed Capacity [MW]')

fig, ax = plt.subplots(subplot_kw={'projection':projection}, figsize=(9,7))

UkIr.plot(ax=ax, zorder=1, transform=plate(), alpha=0.3)
cells.plot(ax=ax, **plot_grid_dict)
layout.plot(ax=ax, transform=plate(), cmap='Reds', vmin=0,
            label='Installed Capacity [MW]')
ax.outline_patch.set_edgecolor('white')
fig.tight_layout()

## Plot Powerproduction

In [ ]:
fig, axes = plt.subplots(len(sites), sharex=True, figsize=(9,4))
powerproduction = cutout.wind('Vestas_V112_3MW', layout=layout,
                              shapes=cells_production.geometry)

powerproduction.to_series().unstack(0).plot(subplots=True, ax=axes)
axes[2].set_xlabel('')
fig.tight_layout()

## Aggregate Power Production to Country Shapes

In [ ]:
from shapely.geometry import Point

fig = plt.figure(figsize=(12,7))
gs = GridSpec(3, 3, figure=fig)

ax = fig.add_subplot(gs[:, 0:2], projection=projection)
gpd.GeoDataFrame(UkIr, columns=['geometry']).assign(color=['1', '2'])\
                .plot(column='color', ax=ax, zorder=1,
                      transform=plate(), alpha=0.6)

sites.assign(geometry=sites.apply(lambda ds: Point(ds.x, ds.y), axis=1))\
     .plot(ax=ax, zorder=2, transform=plate(), color='indianred')

ax.outline_patch.set_edgecolor('white')

powerproduction = cutout.wind('Vestas_V112_3MW', layout=layout.fillna(0),
                              shapes=UkIr).to_series()\
                        .unstack(0).rename_axis(index='', columns='shapes')
ax1 = fig.add_subplot(gs[1, 2])
powerproduction['Ireland'].plot.area(ax=ax1, title='Ireland', color='indianred')
ax2 = fig.add_subplot(gs[2, 2])
powerproduction['United Kingdom'].plot.area(ax=ax2, title='United Kingdom', color='darkgreen')

fig.tight_layout()

## Plot Indicator Matrix
* use seaborn heatmap for plotting

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
indicator_matrix_ir = cutout.indicatormatrix(UkIr)[0]
sns.heatmap(indicator_matrix_ir.toarray().reshape(cutout.shape), cmap='Greens')
fig.savefig(savepath + 'indicator_matrix.png', dpi=100)